In [45]:
import pandas as pd
import numpy as np
import regex
from pathlib import Path

In [46]:
HRSA_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\NSS - Laptop\capstone\data\hrsa\HRSA - 2021 - Cleaned.csv')


In [47]:
# This dataset was created 5-27-2021 by HRSA
# tx_center Count - 254
print(HRSA_raw['OTC Name'].nunique())
# State Count - 48
print(HRSA_raw['Common State Name'].nunique())
# Cities Count - 137
print(HRSA_raw['OTC City'].nunique())
print(HRSA_raw.info())

254
48
137
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809 entries, 0 to 808
Data columns (total 30 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   OPO Name                                               809 non-null    object 
 1   OTC Name                                               809 non-null    object 
 2   OTC Service List                                       809 non-null    object 
 3   OPO Provider #                                         809 non-null    object 
 4   OPO Address                                            809 non-null    object 
 5   OPO City                                               809 non-null    object 
 6   OPO State                                              809 non-null    object 
 7   OPO ZIP                                                809 non-null    object 
 8   OPO Telephone #                        

In [48]:
# Master list of OTC 2021
otc_2021 = HRSA_raw[['OTC Name', 'OTC Address', 'OTC City', 'Common State Name', 'Common State Abbreviation', 'OTC ZIP', 'OTC Longitude', 'OTC Latitude', 'OTC Code']]
otc_2021 = otc_2021.drop_duplicates().reset_index(drop=True)

#OPO master list
opo = HRSA_raw[['OPO Name', 'OPO Address', 'OPO City', 'OPO State', 'OPO ZIP', 'OPO Telephone #', 'OPO Provider #']]
opo = opo.drop_duplicates().reset_index(drop=True)

# Organs transplanted from each OTC
otc_service = HRSA_raw[['OTC Code', 'OTC Service List', 'Organ Transplantation Center Service Type Description']]

# Regions Lists
regions = HRSA_raw[['Common Region Name', 'Common State Name', 'Common Region Code', 'OPO Provider #', 'OTC Code']]
regions = regions.drop_duplicates().reset_index(drop=True)

opo_regions = regions[['Common Region Code', 'OPO Provider #']]
opo_regions = opo_regions.drop_duplicates().reset_index(drop=True)

otc_regions = regions[['Common Region Code', 'OTC Code']]
otc_regions = otc_regions.drop_duplicates().reset_index(drop=True)

regions = regions.drop(columns=['OPO Provider #', 'OTC Code'])
regions = regions.drop_duplicates().reset_index(drop=True)

# OPO key = OPO ZIP
# OTC key = OTC ZIP
hrsa_census_keys = HRSA_raw[['Common State FIPS Code', 'Common State County FIPS Code', 'Common County Name', 'Common City Name with State Abbreviation', 'Common Congressional District Name', 'Common State FIPS Code Congressional District Number', 'Common Postal Code']]
hrsa_census_keys = hrsa_census_keys.drop_duplicates()

srtr_raw['OTC Address'] = srtr_raw['OTC Address'].astype(object)
srtr_raw['Common State Name'] = srtr_raw['Common State Name'].astype(object)
srtr_raw['OTC ZIP'] = srtr_raw['OTC ZIP'].astype(object)
otc_2021['OTC ZIP'] = otc_2021['OTC ZIP'].astype(object)

srtr_raw.merge(otc_2021, how='inner', on='OTC Code')

hospitals = [srtr_raw, otc_2021]
pd.concat(hospitals, join='inner', keys='OTC Code')

In [49]:
srtr_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\NSS - Laptop\capstone\data\otc-2009-raw.csv')

In [74]:
srtr_pending = srtr_raw.loc[srtr_raw['OTC Code'].isnull()].reset_index(drop=True)
srtr_finished = srtr_raw.loc[srtr_raw['OTC Code'].notnull()].reset_index(drop=True)

In [75]:
srtr_pending

,OTC Name,OTC Address,OTC City,Common State Name,Common State Abbreviation,OTC ZIP,OTC Longtitude,OTC Latitude,OTC Code
0,Baptist Medical Center Princeton,NaN,Birmingham,NaN,AL,NaN,NaN,NaN,NaN
1,University of South Alabama Medical Center,NaN,Mobile,NaN,AL,NaN,NaN,NaN,NaN
2,John L. McClellan Veterans Administration Hosp...,NaN,Little Rock,NaN,AR,NaN,NaN,NaN,NaN
3,Banner Good Samaritan Medical Center,NaN,Phoenix,NaN,AZ,NaN,NaN,NaN,NaN
4,Phoenix Regional Medical Center,NaN,Phoenix,NaN,AZ,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
139,Children's Hospital & Regional Medical Center,NaN,Seattle,NaN,WA,NaN,NaN,NaN,NaN
140,Sacred Heart Medical Center,NaN,Spokane,NaN,WA,NaN,NaN,NaN,NaN
141,John L Doyne Hosp,NaN,Milwaukee,NaN,WI,NaN,NaN,NaN,NaN
142,Zablocki Veterans Administration Medical Center,NaN,Milwaukee,NaN,WI,NaN,NaN,NaN,NaN


In [77]:
srtr_pending.to_csv(r'data\srtr_pending.csv')
otc_2021.to_csv(r'data\otc_2021.csv')